In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
import ccxt
from datetime import datetime, timedelta
# import plotly.graph_objects as go
# import inspect 
import os
import ccxt_data_fetch

# pd.set_option('display.max_rows', 50)

In [2]:
exchange = ccxt.binance()

exchange.load_markets()

symbols = exchange.symbols

def ccxt_data(symbol='ETH/USDT', timeframe ='1d', limit=111):

    bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    # must use bars[:-1] because arrgrelextrema will see the last candle wich have not closed yet
    data = pd.DataFrame(bars[:-1], columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    data['Time'] = pd.to_datetime(data['Time'], unit='ms')
    data.set_index('Time', inplace=True)
    
    return data

In [3]:
# data=ccxt_data()


# co = argrelextrema(data['Close'].values, np.greater, order=10)[0]
# df = data.iloc[co]['Close']

# data['ta_rsi'] = ta.rsi(data['Close'], length = 14)
# ta_peak = argrelextrema(data['ta_rsi'].values, np.greater, order=10)[0]
# ta_peaks_df = data.iloc[ta_peak]['ta_rsi']

# peaks = [df, ta_peaks_df]
# peaks_df = pd.concat(peaks, axis=1)
# peaks_df = peaks_df[(peaks_df.Close.notna()) & (peaks_df.ta_rsi.notna())]

# peaks_df

In [4]:
def calc():
    data=ccxt_data()
    symbol='ETH/USDT'

    order=3
    global peaks_df
    global troughs_df

    if len(data) > 14:
    

        data['ta_rsi'] = ta.rsi(data['Close'], length = 14)

        coin_peak = argrelextrema(data['Close'].values, np.greater, order=order)[0]

        coin_peaks_df = data.iloc[coin_peak]['Close']
  

        ta_peak = argrelextrema(data['ta_rsi'].values, np.greater, order=order)[0]
        ta_peaks_df = data.iloc[ta_peak]['ta_rsi']
 

        coin_trough = argrelextrema(data['Close'].values, np.less, order=order)[0]
        coin_trough_df = data.iloc[coin_trough]['Close']

        ta_trough = argrelextrema(data['ta_rsi'].values, np.less, order=order)[0]
        ta_trough_df = data.iloc[ta_trough]['ta_rsi']

        peaks = [coin_peaks_df, ta_peaks_df]
        peaks_df = pd.concat(peaks, axis=1)
        peaks_df = peaks_df[(peaks_df.Close.notna()) & (peaks_df.ta_rsi.notna())]

        troughs = [coin_trough_df, ta_trough_df]
        troughs_df = pd.concat(troughs, axis=1)
        troughs_df = troughs_df[(troughs_df.Close.notna()) & (troughs_df.ta_rsi.notna())]

        if peaks_df.empty:
            # tof=False
            print('empty peaks', symbol)
            return False

        elif troughs_df.empty:
            print('empty troughs', symbol)
            return False
        
        else:
            return peaks_df, troughs_df, True

    else:
        # tof=False
        print('data length less than 14', symbol)
        return False


calc()


In [12]:

n=5
dates_of_peaks = []
if len(peaks_df) > n:
    for i in range(1,n+1):
        diff = peaks_df.diff(i)
        diff.dropna(inplace=True)
        sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
        dates_of_peaks.append(sig.index.values)
else:
    for i in range(1,len(peaks_df)):

        diff = peaks_df.diff(i)
        diff.dropna(inplace=True)
        sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
        dates_of_peaks.append(sig.index.values)

# diff
# peaks_df
# sig
# type(sig)

dates_of_peaks


[array(['2021-12-23T00:00:00.000000000'], dtype='datetime64[ns]'),
 array([], dtype='datetime64[ns]'),
 array([], dtype='datetime64[ns]'),
 array(['2021-11-25T00:00:00.000000000', '2021-11-30T00:00:00.000000000'],
       dtype='datetime64[ns]'),
 array(['2021-11-30T00:00:00.000000000'], dtype='datetime64[ns]')]

In [ ]:

# peaks_df, troughs_df = calc()

def signal_dates(n=9):
    global sell
    global buy
    sell = []
    buy = []

    dates_of_peaks = []
    if len(peaks_df) > n:
        for i in range(1,n+1):
            diff = peaks_df.diff(i)
            diff.dropna(inplace=True)
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_peaks.append(sig.index.values)
    else:
        for i in range(1,len(peaks_df)):

            diff = peaks_df.diff(i)
            diff.dropna(inplace=True)
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_peaks.append(sig.index.values)

    dates_of_troughs = []
    if len(troughs_df) > n:
        for i in range(1,n+1):
            diff = troughs_df.diff(i)
            diff.dropna(inplace=True)
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_troughs.append(sig.index.values)
    else:
        for i in range(1,len(troughs_df)):

            diff = troughs_df.diff(i)
            diff.dropna(inplace=True)
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_troughs.append(sig.index.values)

    
    for list in dates_of_peaks:
        for i in list:
            if str(i) not in sell:
                # i = i + np.timedelta64(4, 'h') # adj sell date
                # sell.append(pd.to_datetime(i))
                sell.append(str(i))

    for list in dates_of_troughs:
        for i in list:
            if str(i) not in buy:
                # i = i + np.timedelta64(4, 'h')
                # buy.append(pd.to_datetime(i))
                buy.append(str(i))
                

    sell.sort()
    buy.sort()
    # print(dates_of_peaks)
    # problem : buy ans sell should be list of datetime***********************

    # sell=pd.to_datetime(sell)
    # buy=pd.to_datetime(buy)

    if not buy or not sell:
        print('empty buy or sell')
        return False
    else:
        return sell, buy, True


